# Demo del taller "Reconocimiento facial"
## DataCup2021
## pip install --upgrade azure-cognitiveservices-vision-face

In [1]:
import glob
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
import time
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person

La siguiente llave y endpoint caducaran en el transcurso de la competencia. Recomendamos utilizar tus propia llave y ENDPOINT

In [2]:
# Key ...........................................
KEY = "b4fa4dc568af40509318c41760728780"
# Endpoint .......................................
ENDPOINT = "https://face-datacup-demo.cognitiveservices.azure.com/"
# Creating a face client ...
face_client = FaceClient(ENDPOINT,CognitiveServicesCredentials(KEY))

In [3]:
#Extrayendo las imagenes de entrenamiento ...
directory = "C:/Users/user/Documents/demo_2_face/notebook+/faces/"
file_list = glob.glob(directory + "*")

In [4]:
file_list

['C:/Users/user/Documents/demo_2_face/notebook+/faces\\hector_faces',
 'C:/Users/user/Documents/demo_2_face/notebook+/faces\\luz_faces',
 'C:/Users/user/Documents/demo_2_face/notebook+/faces\\rick_faces',
 'C:/Users/user/Documents/demo_2_face/notebook+/faces\\test_images']

In [5]:
faces = list()
for i in range(3):
    new_dir = file_list[i]
    new_file_list = glob.glob(new_dir + "/" + "*.jpg")
    faces.append(new_file_list)

In [6]:
faces

[['C:/Users/user/Documents/demo_2_face/notebook+/faces\\hector_faces\\hector.jpg',
  'C:/Users/user/Documents/demo_2_face/notebook+/faces\\hector_faces\\hector_1.jpg',
  'C:/Users/user/Documents/demo_2_face/notebook+/faces\\hector_faces\\hector_3.jpg'],
 ['C:/Users/user/Documents/demo_2_face/notebook+/faces\\luz_faces\\L01096586_4804.jpg',
  'C:/Users/user/Documents/demo_2_face/notebook+/faces\\luz_faces\\L01096586_4807.jpg',
  'C:/Users/user/Documents/demo_2_face/notebook+/faces\\luz_faces\\L01096586_4809.jpg'],
 ['C:/Users/user/Documents/demo_2_face/notebook+/faces\\rick_faces\\rick_1.jpg',
  'C:/Users/user/Documents/demo_2_face/notebook+/faces\\rick_faces\\rick_2.jpg',
  'C:/Users/user/Documents/demo_2_face/notebook+/faces\\rick_faces\\rick_3.jpg']]

# Creando un PersonGroup ...
Esta estructura debe ser entrenada para realizar operaciones de reconocimiento

In [7]:
#Eligiendo un ID...
PERSON_GROUP_ID = "facial_recognition_az_data_cup_group_new"  
#Eligiendo un modelo de reconocimiento facial
model = "recognition_03"

In [8]:
#ESTA LINEA SOLO ES NECESARIA CORRERLA UNA SOLA VEZ PARA CREAR UN GRUPO CON ID UNICO (Descomentar si se desea crear un nuevo grupo)

face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name="grupo_demo_1", recognition_model= model) 

In [9]:
#face_client.person_group.delete(PERSON_GROUP_ID)     #LINEA DE CÓDIGO PARA ELIMINAR UN GRUPO DE PERSONAS 

In [10]:
# Definiendo las personas que estaran dentro del grupod de entrenamiento 
names = [("Héctor","L03143082" ),("Ricardo", "A01066515"),("Luz", "L01096586")]
for name, nomina in names:
    face_client.person_group_person.create(PERSON_GROUP_ID, name = name, user_data= nomina)


In [11]:
#Desplegando a las personas dentro de un grupo ...
people = face_client.person_group_person.list(PERSON_GROUP_ID)
ids = list()
for  person in people:
    print(person.user_data,person.name," ",person.person_id)

    if person.name == "Héctor":
       hector_id = person.person_id
    elif person.name == "Luz":
        luz_id = person.person_id
    elif person.name == "Ricardo":
        rick_id = person.person_id



A01066515 Ricardo   8e68661a-8f4d-48d6-bc54-d876dccf9278
L03143082 Héctor   d6cf75f2-8445-45ab-9c06-fdcfa9a08a09
L01096586 Luz   e0794cd1-7d29-4409-a5e4-9be5da84de85


In [12]:
rick_id

'8e68661a-8f4d-48d6-bc54-d876dccf9278'

# Agregando rostros a cada persona 

In [13]:
for count, files in enumerate(faces):
    for face in  files:
        image = open(face, 'r+b')
        if count == 0:
            face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, hector_id,image)
        elif count == 1:
            face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, luz_id,image)
        elif count == 2:
            face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, rick_id,image)    


# Entrenando el modelo

In [14]:
# Training the person group ...
face_client.person_group.train(PERSON_GROUP_ID)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        sys.exit('Training the person group has failed.')
    time.sleep(5)

Training status: running.

Training status: succeeded.



# Operaciones de detección e identificación

In [15]:

test_image = glob.glob("test_2.jpg")
image = open(test_image[0], 'r+b')

time.sleep (60)
#Caras detectadas
face_ids = list()
faces = face_client.face.detect_with_stream(image, detection_model='detection_03',recognition_model= 'recognition_03')
for face in faces:
    face_ids.append(face.face_id)


In [16]:
 face_client.face.identify(face_ids, PERSON_GROUP_ID)

In [17]:
#Operacion de identificacion
results = face_client.face.identify(face_ids, PERSON_GROUP_ID)
if not results:
    print("No existen resultados :(")
for person in results:
    if len(person.candidates) > 0:
        person_id_ = person.candidates[0].person_id
        indi_identificado = face_client.person_group_person.get(PERSON_GROUP_ID,person_id_)
        print(' {} con matricula {} es identificado en  {} con una certeza de {}'.format(indi_identificado.name,indi_identificado.user_data,
         os.path.basename(image.name), person.candidates[0].confidence))
    else:
        print('No se identificó a la persona :(' )

 Ricardo con matricula A01066515 es identificado en  test_2.jpg con una certeza de 0.92123


In [18]:
face_client.person_group.delete(PERSON_GROUP_ID) #Borrar el grupo despues de correr una vez el script